# MODEL REGISTRY WITH MLFLOW

## SET UP MLFLOW EXPERIMENT TRACKING LOCAL HOST

In [1]:
import mlflow
from mlflow.tracking import MlflowClient

mlflow_tracking_uri = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=mlflow_tracking_uri)

### List all experiments and inspect some of them

In [2]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='taxi_trip_prediction-experiment', tags={'mlflow.note.content': '**TAXI TRIP DURATION PREDICTION EXPERIMENT**\n'
                         '\n'
                         'Model selection:\n'
                         '\n'
                         '1. Linear regression\n'
                         '2. Lasso regression\n'
                         '3. Xgboost with hyperopt\n'
                         '4. RandomForrestRegressor\n'
                         '5. GradientBoostingRegressor\n'
                         '6. ExtraTreesRegressor\n'
                         '7. LinearSVR\n'
                         '\n'}>]

In [31]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids = '1',
    filter_string = "metrics.rmse < 7 and tags.model != 'hyperopt'",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 10,
    order_by = ['metric.rmse ASC']
)

In [32]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 3cd71efdccd4468f85d0278e8b062bea, rmse: 6.3616
run id: a03610faac554893b4ac256794cbb7b0, rmse: 6.7423
run id: fe6216abaa3d454087b0d927126121a9, rmse: 6.9098
run id: 3512dbc2a311452389795595cd471c17, rmse: 6.9227


In [43]:
model_list = [run.info.run_id for run in runs]
model_list[0]

'3cd71efdccd4468f85d0278e8b062bea'

### Interacting with the Model Registry

- set tracking uri
- define run id
- define model uri
- register model with register_model method
    - parameter
        1. model uri
        2. model name
        
This section use the **MlflowClient** instance to:

    1. Register a new version for the experiment "taxi_trip_regressor"
    2. Retrieve the latests versions of the model "taxi_trip_regressor" and check that a new version in this case "3" was created.
    3. Transition the version "3" to "Staging" and adding annotations to it.

**Registry model with python**

In [34]:
mlflow.set_tracking_uri(mlflow_tracking_uri)

for run_id in model_list:
    model_uri = f"runs:/{run_id}/model"
    mlflow.register_model(model_uri=model_uri, 
                          name="nyc_taxi_trip_regressor",
                          await_registration_for=120)

Registered model 'nyc_taxi_trip_regressor' already exists. Creating a new version of this model...
2022/07/13 11:05:25 INFO mlflow.tracking._model_registry.client: Waiting up to 120 seconds for model version to finish creation.                     Model name: nyc_taxi_trip_regressor, version 3
Created version '3' of model 'nyc_taxi_trip_regressor'.
Registered model 'nyc_taxi_trip_regressor' already exists. Creating a new version of this model...
2022/07/13 11:05:25 INFO mlflow.tracking._model_registry.client: Waiting up to 120 seconds for model version to finish creation.                     Model name: nyc_taxi_trip_regressor, version 4
Created version '4' of model 'nyc_taxi_trip_regressor'.
Registered model 'nyc_taxi_trip_regressor' already exists. Creating a new version of this model...
2022/07/13 11:05:25 INFO mlflow.tracking._model_registry.client: Waiting up to 120 seconds for model version to finish creation.                     Model name: nyc_taxi_trip_regressor, version 5
Cre

In [37]:
model_name = "nyc_taxi_trip_regressor"
latest_versions = client.get_latest_versions(name=model_name)

model_none_versions = list()
model_stg_versions = list()

for version in latest_versions:
    
    if version.current_stage == "None":
        print(f"version: {version.version}, stage: {version.current_stage}")
        model_none_versions.append(version.version)
    elif version.current_stage == "Staging":
        print(f"version: {version.version}, stage: {version.current_stage}")
        model_stg_versions.append(version.version) 

version: 2, stage: Staging
version: 6, stage: None


In [38]:
print(f"new_model: {model_none_versions}")
print(f"model on staging: {model_stg_versions}")

new_model: [6]
model on staging: [2]


In [39]:
selected_model = model_none_versions[-1]
model_stages = ["Staging", "Production", "Archived", "None"]
new_stage = model_stages[0]

client.transition_model_version_stage(
    name=model_name,
    version=selected_model,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1657685125710, current_stage='Staging', description=None, last_updated_timestamp=1657686073311, name='nyc_taxi_trip_regressor', run_id='3512dbc2a311452389795595cd471c17', run_link=None, source='./mlruns/1/3512dbc2a311452389795595cd471c17/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.

Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.



In [40]:
from sklearn.metrics import mean_squared_error
import pandas as pd

def read_dataframe(filename):
    df = pd.read_parquet(filename)
        
    df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
    df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
    
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df.duration.apply(lambda x: x.total_seconds() / 60)
    
    df = df[(df['duration'] >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    
    return dv.transform(train_dicts)

def test_model(name, X_test, y_test, stage_name=None, version_no=None, model_name=None, stage=False, version=False, runs=False):
    
    '''
    Choose how to load model
    - version
    - stage
    - model flavor
    '''
    
    if version:
        model = mlflow.pyfunc.load_model(f"models:/{name}/{version}")
        y_pred = model.predict(X_test)
    elif runs:
        model = mlflow.pyfunc.load_model(f"runs:/{name}/{model_name}")
        y_pred = model.predict(X_test)
    elif stage:
        model = mlflow.pyfunc.load_model(f"models:/{name}/{stage_name}")
        y_pred = model.predict(X_test)
    
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}
    

In [44]:
mlflow.set_tracking_uri(mlflow_tracking_uri)

run_id = model_list[0]
model_name = "nyc_taxi_trip_regressor"

model_stages = ["Staging", "Production", "Archived", "None"]

In [41]:
test_path = './data/green_tripdata_2021-03.parquet'

In [45]:
df_test = read_dataframe(test_path)

In [46]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/Users/premsurawut/_Github/MLOps/main_course/course-mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [47]:
import pickle

with open('preprocessor/preprocessor.b', 'rb') as f_in:
    dv = pickle.load(f_in)

In [48]:
X_test = preprocess(df_test, dv)
X_test

<80372x13221 sparse matrix of type '<class 'numpy.float64'>'
	with 153356 stored elements in Compressed Sparse Row format>

In [49]:
target = 'duration'
y_test = df_test[target].values
y_test

array([ 8.35      ,  5.23333333, 20.33333333, ...,  9.        ,
       25.        ,  9.        ])

In [51]:
%time 

prod = test_model(name=model_name, X_test=X_test, y_test=y_test, stage_name="Production", stage=True) 

CPU times: user 1 µs, sys: 4 µs, total: 5 µs
Wall time: 10 µs


In [52]:
%time 
new_model = test_model(name=run_id, X_test=X_test, y_test=y_test, model_name='models_xgboost', runs=True)

CPU times: user 2 µs, sys: 4 µs, total: 6 µs
Wall time: 14.1 µs
[11:30:05] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [53]:
print("COMPARING BETWEEN THE CURRENT MODEL IN PRODUCTION AND NEW MODEL IN STAGING")
print(f"Prodcuction: Linear regression, {prod}")
print(f"Staging : XGBooster, {new_model}")

COMPARING BETWEEN THE CURRENT MODEL IN PRODUCTION AND NEW MODEL IN STAGING
Prodcuction: Linear regression, {'rmse': 7.720735231839729}
Staging : XGBooster, {'rmse': 6.293845077375141}


### PUT NEW MODEL TO THE PRODUCTION

In [54]:
def update_new_model(model_name, new_prod_ver, stage="Production", archive_existing_versions=False):
    client.transition_model_version_stage(
        name=model_name,
        version=new_prod_ver,
        stage=stage,
        archive_existing_versions=archive_existing_versions
    )
    return "Uploaded New model to Production successfully"

update_new_model(model_name, model_none_versions[-1], archive_existing_versions=True)

'Uploaded New model to Production successfully'

### UPDATE MODEL STATUS

In [56]:
def update_model_metadata(model_name, new_prod_ver, old_prod_ver):
    
    from datetime import datetime
    date = datetime.today().date()
    
    
    client.update_model_version(
        name=model_name,
        version=new_prod_ver,
        description=f"Uploaded model to production on {date}")
    
    client.update_model_version(
        name=model_name,
        version=old_prod_ver,
        description=f"Transition model to archive on {date}")
    
    return f"Updated model version Succesfully"

In [57]:
update_model_metadata(model_name, 6, 1)

'Updated model version Succesfully'